In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from datetime import datetime
import cv2
from joblib import Parallel, delayed



# Load the data with the correct header
labels = pd.read_excel('/Users/thanmai/Desktop/DAQUIP/dataset/labels.xlsx', header=1)

# Rename the columns
labels.columns = ['Image no.', 'Date', 'Time', 'Place', 'NowCastConc.', 'PM2.5', 'AQI Category', 'Raw Conc.', 'Conc. Unit']

# Verify the column names
print("Columns in the DataFrame after renaming:", labels.columns)

# Function to parse dates with multiple formats
def parse_date(date_str):
    if isinstance(date_str, datetime):
        return date_str
    for fmt in ('%d.%m.%Y', '%Y-%m-%d'):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    raise ValueError(f"Date format for {date_str} not recognized")

# Convert Date
labels['Date'] = labels['Date'].apply(parse_date)

# Function to parse times with multiple formats
def parse_time(time_str):
    if isinstance(time_str, datetime):
        return time_str
    for fmt in ('%I.%M%p', '%I:%M:%S%p', '%H:%M:%S'):
        try:
            return datetime.strptime(time_str, fmt).time()
        except ValueError:
            continue
    raise ValueError(f"Time format for {time_str} not recognized")

# Ensure 'Time' column is converted to strings before parsing
labels['Time'] = labels['Time'].astype(str)
labels['Time'] = labels['Time'].apply(parse_time)

# Extract useful features from Date and Time
labels['Hour'] = labels['Time'].apply(lambda x: x.hour)
labels['DayOfWeek'] = labels['Date'].apply(lambda x: x.weekday())
labels['Month'] = labels['Date'].apply(lambda x: x.month)


# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_place = encoder.fit_transform(labels[['Place']])
encoded_aqi_category = encoder.fit_transform(labels[['AQI Category']])

# Function definitions for feature extraction
def extract_dark_channel(image):
    min_channel = np.min(image, axis=2)
    return min_channel

def estimate_transmission(dark_channel, omega=0.95):
    transmission = 1 - omega * dark_channel
    return transmission

def get_sky_color(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    sky_color = np.mean(lab, axis=(0, 1))
    return sky_color

def power_spectrum_slope(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    slope = np.mean(magnitude_spectrum)
    return slope

def calculate_contrast(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast = np.sqrt(np.mean((gray - np.mean(gray))**2))
    return contrast

def calculate_normalized_saturation(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    saturation = hsv[..., 1]
    normalized_saturation = (saturation - np.min(saturation)) / (np.max(saturation) - np.min(saturation))
    histogram = np.histogram(normalized_saturation, bins=10, range=(0, 1))[0]
    return histogram

# Define the function for feature extraction from images
def extract_features(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Error reading image: {image_path}")
        
        dark_channel = extract_dark_channel(image)
        transmission = estimate_transmission(dark_channel)
        sky_color = get_sky_color(image)
        slope = power_spectrum_slope(image)
        contrast = calculate_contrast(image)
        normalized_saturation = calculate_normalized_saturation(image)

        features = [
            float(np.mean(dark_channel)),
            float(np.mean(transmission)),
            float(np.mean(sky_color)),
            float(slope),
            float(contrast),
            *normalized_saturation.tolist()
        ]
        return features
    except Exception as e:
        print(e)
        return [np.nan] * 7

# Parallel feature extraction from images
num_cores = -1  # Use all available CPU cores
X_images = Parallel(n_jobs=num_cores)(delayed(extract_features)(f'/Users/thanmai/Desktop/DAQUIP/dataset/{row["Image no."]}.jpg') for index, row in labels.iterrows())

# Convert extracted features to numpy array and handle NaNs
X_images = np.array(X_images, dtype=np.float64)
X_images = np.nan_to_num(X_images)  # Replace NaNs with 0s or other appropriate value

# Combine all features
X = np.hstack((X_images, encoded_place, encoded_aqi_category, labels[['NowCastConc.', 'Raw Conc.', 'Hour', 'DayOfWeek', 'Month']].values))
y = labels['PM2.5'].values

# Normalize/Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Ensure there are no NaNs in the combined dataset
X = np.nan_to_num(X)
y = np.nan_to_num(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)

# Save the model
model.save('/Users/thanmai/Desktop/DAQUIP/models/cnn_model.h5')

# Predict and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Model Mean Squared Error: {mse}")

# Calculate R² score
r2 = r2_score(y_test, y_pred)
print(f"Model R²: {r2}")

# Calculate baseline model
baseline_pred = np.mean(y_train)
baseline_mse = mean_squared_error(y_test, [baseline_pred] * len(y_test))
print(f"Baseline MSE: {baseline_mse}")


Columns in the DataFrame after renaming: Index(['Image no.', 'Date', 'Time', 'Place', 'NowCastConc.', 'PM2.5',
       'AQI Category', 'Raw Conc.', 'Conc. Unit'],
      dtype='object')


/Users/thanmai/Desktop/DAQUIP/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Premature end of JPEG file
Premature end of JPEG file
/Users/thanmai/Desktop/DAQUIP/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 25050.5762 - mse: 25050.5762 - val_loss: 25375.6699 - val_mse: 25375.6699
Epoch 2/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 24094.7695 - mse: 24094.7695 - val_loss: 23235.3652 - val_mse: 23235.3652
Epoch 3/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step - loss: 20955.4766 - mse: 20955.4766 - val_loss: 17770.3066 - val_mse: 17770.3066
Epoch 4/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: 14899.0645 - mse: 14899.0645 - val_loss: 9024.4453 - val_mse: 9024.4453
Epoch 5/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - loss: 6313.0298 - mse: 6313.0298 - val_loss: 2898.8701 - val_mse: 2898.8701
Epoch 6/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - loss: 2196.0498 - mse: 2196.0498 - val_loss: 1573.8494 - val_mse: 1573.8494
Epoch 7/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - loss: 1501.8947 - mse: 1501.8947 - val_loss: 1297.6997 - val_mse: 1297.6997
Epoch 8/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - loss: 1323.8663 - mse: 1

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Model Mean Squared Error: 527.5411525809114
Model R²: 0.8104992508888245
Baseline MSE: 2785.1009443398357


In [10]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Example of feature extraction from date and time
labels['Day'] = labels['Date'].dt.day
labels['Month'] = labels['Date'].dt.month
labels['Hour'] = labels['Time'].dt.hour

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels['PM2.5'], test_size=0.2, random_state=42)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

# Evaluate the model
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Random Forest MSE: {mse}")
print(f"Random Forest R²: {r2}")

# Ensemble with Gradient Boosting
gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)
gbr.fit(X_train, y_train)

y_pred_gbr = gbr.predict(X_test)
mse_gbr = mean_squared_error(y_test, y_pred_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)
print(f"Gradient Boosting MSE: {mse_gbr}")
print(f"Gradient Boosting R²: {r2_gbr}")
